In [1]:
# -*- coding:UTF-8 -*-

    
from corpus import *
from vsm import *
from vsm.spatial import KL_div
import numpy as np


k = topic_range[0]
v = lda_v[k]
c.words

# train the model and create a TfViewer object
tf = TF(c, context_type)
tf.train()
tf_v = TfViewer(c, tf)

# print the most frequent terms in the document
# remember that IPython automatically prints the last cell of a document
tf_v.coll_freqs()




#v.topics()

#v.doc_topics(v.labels[:3])


#v.dist(v.labels[0], v.labels[1])
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import randn
import numpy as np
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname='/home/hongliang/Downloads/msyh.ttf/msyh.ttf')
import sys
reload(sys)
kongzi = []
mengzi = []
xunzi =[]

count = 0
for label in v.labels[:]:
    if '孟子' in label:
        mengzi.append(count)
    elif '论语' in label:
        kongzi.append(count)
    elif '荀子' in label:
        xunzi.append(count)
    count += 1



Running from notebook, using serial load function.
[20, 40, 60, 80, 100]
/home/inpho/inpho/kmx/models/kmx-freq5-freq5-N2523342-LDA-K{0}-document-2000.npz
Loading LDA data from /home/inpho/inpho/kmx/models/kmx-freq5-freq5-N2523342-LDA-K20-document-2000.npz


In [9]:
def similarity(c1,c2):
    A = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            A[m][n]= v.dist(v.labels[c1[m]],v.labels[c2[n]]) 
    print "while k quals",k,',','JSD goes',np.mean(A)
    return np.mean(A)

def kl_similarity(c1,c2):
    A = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            A[m][n]= v.dist(v.labels[c1[m]],v.labels[c2[n]], dist_fn=KL_div) 
    print "while k quals",k,',','first to second KLD goes',np.mean(A)
    
    B = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            B[m][n]= v.dist(v.labels[c2[n]],v.labels[c1[m]], dist_fn=KL_div) 
    print "while k quals",k,',','second to first KLD goes',np.mean(B)    
    return np.mean(A),np.mean(B)

def perplexity(theta, corpus, v):
    a = (v.phi[corpus] * theta).sum(axis=1)
    a = np.log2(a).sum()
    return a / len(corpus)

In [23]:
corpus = v.corpus.corpus
d_value = dict([])
for i in range(5):
    k = topic_range[i]
    v = lda_v[k]
    theta = v.aggregate_doc_topics(all_ids)
    theta = theta[np.argsort(theta['i'])]
    theta = np.array(theta['value'])
    P = perplexity(theta, corpus, v)
    d_value[str(k)] = []
    print 'kongzi mengzi JSD'
    d_value[str(k)].append(similarity(kongzi,mengzi))
    print 'kongzi xunzi JSD'
    d_value[str(k)].append(similarity(kongzi,xunzi))
    print 'xunzi mengzi JSD'
    d_value[str(k)].append(similarity(xunzi,mengzi))
    print 'kongzi mengzi KLD'
    d_value[str(k)].append(kl_similarity(kongzi,mengzi)[0])
    d_value[str(k)].append(kl_similarity(kongzi,mengzi)[1])
    print 'kongzi xunzi KLD'
    d_value[str(k)].append(kl_similarity(kongzi,xunzi)[0])
    d_value[str(k)].append(kl_similarity(kongzi,xunzi)[1])
    print 'xunzi mengzi KLD'
    d_value[str(k)].append(kl_similarity(xunzi,mengzi)[0])
    d_value[str(k)].append(kl_similarity(xunzi,mengzi)[1])
    d_value[str(k)].append(P)
    


In [24]:
print d_value

In [25]:
topicname = []
topicname.append('kongzi mengzi JSD')
topicname.append('kongzi xunzi JSD')
topicname.append ('xunzi mengzi JSD')
topicname.append('kongzi mengzi KLD')
topicname.append('mengzi kongzi KLD')
topicname.append('kongzi xunzi KLD')
topicname.append('xunzi kongzi KLD')
topicname.append('xunzi mengzi KLD')
topicname.append('mengzi xunzi KLD')
topicname.append('perplexity')

In [26]:
from pandas import Series,DataFrame
import pandas as pd
#data = {"Topics":top10topics[:10], "Corpus Weight":top10topicsweight[:10], "Topic Name":[,,,,,,,,,],"Topic Keywords in Descending Order of Weight":chunks(word,10)} 

data3 = {"name":topicname,"20":d_value['20'],"40":d_value['40'],"60":d_value['60'],"80":d_value['80'],"100":d_value['100'],}
f3 = DataFrame(data3,columns=['name','20','40','60','80','100'])
f3

,name,20,40,60,80,100
0,kongzi mengzi JSD,0.486841,0.642040,0.708402,0.749273,0.866783
1,kongzi xunzi JSD,0.596306,0.747565,0.822088,0.838763,0.849756
2,xunzi mengzi JSD,0.587285,0.708401,0.781842,0.853024,0.892841
3,kongzi mengzi KLD,1.744121,3.234609,5.226748,5.924545,9.194358
4,mengzi kongzi KLD,3.557761,5.783952,6.610855,7.327600,8.916091
5,kongzi xunzi KLD,3.277708,4.698098,8.721637,10.000375,9.661511
6,xunzi kongzi KLD,5.623798,8.065141,8.492631,8.491108,8.819306
7,xunzi mengzi KLD,4.302596,6.891937,7.614338,9.924586,10.390576
8,mengzi xunzi KLD,4.462231,5.892509,8.193616,9.648914,10.845593
9,perplexity,-10.309580,-10.309552,-10.309571,-10.309587,-10.309582


## my method

In [ ]:
# from corpus import *
# import numpy as np

# def perplexity(theta, corpus, v):
#    a = (v.phi[corpus] * theta).sum(axis=1)
#    a = np.log2(a).sum()
#    return a / len(corpus)

# v = lda_v[80]
# theta = v.aggregate_doc_topics(all_ids)
# TMP = []
# for a in theta:
#    TMP.append(a[1]) 
# theta = np.array(TMP)
# print theta
# corpus = v.corpus.corpus
# P = perplexity(theta, corpus, v)
# print P

## each to whole

In [ ]:
# def perplexity(theta, corpus, v):
#     a = (v.phi[corpus] * theta).sum(axis=1)
#     a = np.log2(a).sum()
#     return a / len(corpus)

# v = lda_v[80]    
# perplexity_num = []
# theta_all = []
# documents = v.corpus.view_contexts('document')
# for doc_id,corpus in zip(all_ids,documents):
#     theta = v.doc_topic_matrix(doc_id)
#     print theta
#     theta_all.append(theta)
#     perplexity_num.append(perplexity(theta,corpus,v))
# perplexity_num = np.array(perplexity_num)
# theta_all = np.array(theta_all)
# print 'each document way',np.mean(perplexity_num)


### Jaimie's Final method

In [ ]:
# from corpus import *
# import numpy as np

# def perplexity(theta, corpus, v):
#     a = (v.phi[corpus] * theta).sum(axis=1)
#     a = np.log2(a).sum()
#     return a / len(corpus)

# v = lda_v[80]
# corpus = v.corpus.corpus

# # v.aggregate_doc_topics returns an ndarray instead of just values.
# theta = v.aggregate_doc_topics(all_ids)
# print 'theta 1 step',theta
# theta = theta[np.argsort(theta['i'])]
# print 'theat2 step',theta
# theta = np.array(theta['value'])
# print 'theta3 step',theta

# P = perplexity(theta, corpus, v)
# print 'whole corpus',P
